In [2]:
# basic packages 
import numpy as np
import pandas as pd
import os
import math
import time
import ast
import time
import argparse
from matplotlib import pyplot as plt

#for AE 
#kernel = Python 3
#import torch
#import torch.nn as nn
#import torch.nn.parallel
#import torch.optim as optim
#import torch.utils.data

#from torch.nn import functional
#from torch.autograd import Variable

#from sklearn.model_selection import train_test_split

#import timeit
#Source
#VAE keras: https://keras.io/examples/generative/vae/
#VAE CF: https://github.com/kilolgupta/Variational-Autoencoders-Collaborative-Filtering



Source
* [VAE keras](https://keras.io/examples/generative/vae/) (for 3d image data inputs)
* [VAE CF](https://github.com/kilolgupta/Variational-Autoencoders-Collaborative-Filtering)
* [VAE for CF](https://github.com/dawenl/vae_cf/blob/master/VAE_ML20M_WWW2018.ipynb)

[Activation function discussion](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6)

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
#data processing
#get data based on census tract
#based on downtown Seattle
#test data
os.chdir('G:/My Drive/2021/Bias/')

test_matrice_reverse = pd.read_csv('test_matrice_reverse.csv', header=None).to_numpy()
Zero_index = pd.read_csv('Zero_index.csv', header=None).to_numpy()
#create a 0-1 value matrice
test_matrice_reverse_01 = np.where(test_matrice_reverse[:, 1:]>0, 1, 0)
#os.listsdir()

#land use data 
#obs
#obs_01
#zero index
'''
landuse_input_path = 'G:/My Drive/2021/Bias/census_block_data/'
land_use_obs = pd.read_csv('obs_landuse.csv', header=None).to_numpy()
land_use_01 =  pd.read_csv('obs01_landuse.csv', header=None).to_numpy()

land_use_zero_index = pd.read_csv('zero_index_landuse.csv',header=None).to_numpy()'''

"\nlanduse_input_path = 'G:/My Drive/2021/Bias/census_block_data/'\nland_use_obs = pd.read_csv('obs_landuse.csv', header=None).to_numpy()\nland_use_01 =  pd.read_csv('obs01_landuse.csv', header=None).to_numpy()\n\nland_use_zero_index = pd.read_csv('zero_index_landuse.csv',header=None).to_numpy()"

In [5]:
#sparsity of data in the paper
20000263/(27278*138493)

#sparsity in the cuebiq data
np.count_nonzero(test_matrice_reverse[:, 1:])/(7767*3456)

0.004185859929712125

In [36]:
import os, sys
import pandas as pd
import numpy as np
import random
import pickle
from scipy import sparse

In [ ]:
#VAE CF: https://github.com/kilolgupta/Variational-Autoencoders-Collaborative-Filtering
test_path = 'G:/My Drive/2021/Bias/recsys_test/'
os.chdir(test_path)

ratings = pd.read_csv('ml-20m/ratings.csv')
movies = pd.read_csv('ml-20m/movies.csv')
links = pd.read_csv('ml-20m/links.csv')

movie_id = np.unique(movies['movieId'])
ratings = ratings[ratings['movieId'].isin(movie_id)]

In [ ]:
number_of_users = len(unique_user_ids)
print("total number of users: ", number_of_users)

number_of_movies = len(unique_movie_ids)
print("total number of movies: ", number_of_movies)

# split users into training, validation and test
val_user_ids = []
test_user_ids = []
train_user_ids = []

for i in range(10000):
    val_user_ids.append(unique_user_ids[i]) # first 10k after shuffling keys

for i in range(10000, 20000):
    test_user_ids.append(unique_user_ids[i]) # next 10k after shuffling keys

for i in range(20000, number_of_users): # all the remaining form training data
    train_user_ids.append(unique_user_ids[i])


# creating a movieId and userId to index dictionary for creating train_data ndarray
movie2id = {}
movie2id = dict((mid, i) for (i, mid) in enumerate(unique_movie_ids))


#------------------------------------------------------------------------------------------------------------------------------------

print("creating training data....")

user2id = {}
user2id = dict((uid, i) for (i, uid) in enumerate(train_user_ids))


rows = []
cols = []
for u_id in train_user_ids:
    print("train-",user2id[u_id])
    m_ids = raw_data[(raw_data.userId == u_id) & (raw_data.rating > 3.5)]['movieId'].tolist()
    movie_indexes = [movie2id[m] for m in m_ids]
    rows.extend([user2id[u_id] for i in range(len(m_ids))])
    cols.extend(movie_indexes)

# pickle.dump(rows, open("rows.file", "wb"))
# pickle.dump(cols, open("cols.file", "wb"))


# creating a sparse matrix with no_of_train_users X movies for training, binarized feedback
# rows and cols should be of same length
train_data = sparse.csr_matrix((np.ones_like(rows),(np.array(rows), np.array(cols))), dtype='float64', shape=(len(train_user_ids), number_of_movies))

# dumping variable to load later for use in VAE code
pickle.dump(train_data, open("train_data.file", "wb"))
print("number of training users: ", len(train_user_ids))


#-----------------------------------------------------------------------------------------------------------------------------------

print("creating test data....")
test_user2id = {}
test_user2id = dict((uid, i) for (i, uid) in enumerate(test_user_ids))

test_rows = []
test_cols = []
for u_id in test_user_ids:
    print("test-", test_user2id[u_id])
    m_ids = raw_data[(raw_data.userId == u_id) & (raw_data.rating > 3.5)]['movieId'].tolist()
    movie_indexes = [movie2id[m] for m in m_ids]
    test_rows.extend([test_user2id[u_id] for i in range(len(m_ids))])
    test_cols.extend(movie_indexes)

test_data = sparse.csr_matrix((np.ones_like(test_rows),(np.array(test_rows), np.array(test_cols))), dtype='float64', shape=(len(test_user_ids), number_of_movies))
pickle.dump(test_data, open("test_data.file", "wb"))
print("number of test users: ", len(test_user_ids))

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------


print("creating validation data")
val_user2id = {}
val_user2id = dict((uid, i) for (i, uid) in enumerate(val_user_ids))

val_rows = []
val_cols = []
for u_id in val_user_ids:
    print("val-", val_user2id[u_id])
    m_ids = raw_data[(raw_data.userId == u_id) & (raw_data.rating > 3.5)]['movieId'].tolist()
    movie_indexes = [movie2id[m] for m in m_ids]
    val_rows.extend([val_user2id[u_id] for i in range(len(m_ids))])
    val_cols.extend(movie_indexes)

val_data = sparse.csr_matrix((np.ones_like(val_rows),(np.array(val_rows), np.array(val_cols))), dtype='float64', shape=(len(val_user_ids), number_of_movies))
pickle.dump(val_data, open("val_data.file", "wb"))
print("number of validation users: ", len(val_user_ids))

In [ ]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

# encoder/decoder network size
batch_size=500
original_dim=26621 # number of movies
intermediate_dim=600
latent_dim=200
nb_epochs=20
epsilon_std=1.0


# encoder network
x=Input(batch_shape=(batch_size,original_dim))
h=Dense(intermediate_dim, activation='tanh')(x)
z_mean=Dense(latent_dim)(h)
z_log_var=Dense(latent_dim)(h)


# sampling from latent dimension for decoder/generative part of network
def sampling(args):
    _mean,_log_var=args
    epsilon=K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., std=epsilon_std)
    return _mean+K.exp(_log_var/2)*epsilon

z= Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# decoder network
h_decoder=Dense(intermediate_dim, activation='tanh')
x_bar=Dense(original_dim,activation='softmax') # this should be softmax right?
h_decoded = h_decoder(z)
x_decoded = x_bar(h_decoded)

# build and compile model
vae = Model(x, x_decoded)
def vae_loss(x,x_bar):
    reconst_loss=original_dim*objectives.binary_crossentropy(x, x_bar)
    kl_loss= -0.5*K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return kl_loss + reconst_loss

vae.compile(optimizer='adam', loss=vae_loss)


x_train = pickle.load( open( "train_data.file", "rb" ) )
print("number of training users: ", x_train.shape[0])

x_val = pickle.load( open( "val_data.file", "rb" ) )
x_val = x_val.todense()

def nn_batch_generator(x, y, batch_size, samples_per_epoch):
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    x =  x[shuffle_index, :]
    y =  y[shuffle_index, :]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        x_batch = x[index_batch,:].todense()
        y_batch = y[index_batch,:].todense()
        counter += 1
        yield (np.array(x_batch),np.array(y_batch))
        if (counter >= number_of_batches):
            counter=0


weightsPath = "./tmp/weights.hdf5"
checkpointer = ModelCheckpoint(filepath=weightsPath, verbose=1, save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# sending complete training data and shuffle flag will shuffle so that each user comes atleast once in training because of multiple epochs
vae.fit_generator(nn_batch_generator(x_train, x_train, batch_size, 118000), samples_per_epoch=118000, nb_epoch=nb_epochs, 
    validation_data=(x_val, x_val), callbacks=[checkpointer, reduce_lr, history])

print("training losses over epochs")
print(history.losses)

print("validation losses over epochs")
print(history.val_losses)